In [ ]:
import openslide

import torch
from torch import nn
from torchvision import transforms, models

import os
import copy
from tqdm import tqdm
import pandas as pd
import numpy as np
from PIL import Image
import cv2

# Tissue map

## Load model

In [ ]:
model = models.resnet50(pretrained=True)

model.fc = nn.Sequential(
    nn.Linear(2048, 1024),
    nn.Dropout(0.5),
    nn.Linear(1024, 9)
)

model.load_state_dict(torch.load('./resnet50_weights.pth'))
model = model.to('cuda')
model.eval()

## Transform

In [ ]:
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

## WSI tile classification

In [ ]:
TCGA_COAD_PATH = './TCGA_COAD'
foldername = os.listdir(TCGA_COAD_PATH)

In [ ]:
pathology_list = pd.read_csv('./pathology_list.csv', index_col = 'pathology')
pathology_list_index = list(pathology_list.index)
#print(len(pathology_list_index))

In [ ]:
TILE = []
PATHOLOGY = []
WIDTH_TILE = []
HEIGHT_TILE = []
X_TILE = []
Y_TILE = []
TISSUE = []

for i in tqdm(range(0, len(foldername))):
    if os.path.isdir(os.path.join(TCGA_COAD_PATH, foldername[i])):
        filename = os.listdir(os.path.join(TCGA_COAD_PATH, foldername[i]))
        
        for j in range(0, len(filename)):
            if filename[j][-3:len(filename[j])] == "svs":
                if filename[j][0:23] not in pathology_list_index:
                    continue
                
                slide = openslide.OpenSlide(os.path.join(TCGA_COAD_PATH, foldername[i], filename[j]))
                try:
                    magnification = int(slide.properties['aperio.AppMag'])
                except:
                    magnification = 20
                factor = int(magnification/20)
                    
                [W, H] = slide.level_dimensions[0]
                w = int(W*(20/magnification))
                h = int(H*(20/magnification))
                tile_number_width = w//224
                tile_number_height = h//224
                print(f'{filename[j][0:23]}: {tile_number_width}*{tile_number_height}')
                
                num = 0
                for x in range(0, tile_number_width):
                    for y in range(0, tile_number_height):
                        num = num + 1
                        TILE.append(filename[j][0:23] + '_' + str(num))
                        PATHOLOGY.append(filename[j][0:23])
                        WIDTH_TILE.append(tile_number_width)
                        HEIGHT_TILE.append(tile_number_height)
                        X_TILE.append(x+1)
                        Y_TILE.append(y+1)
                        
                        location = (x*224*factor, y*224*factor)
                        crop = slide.read_region(location = location, level = 0, size = (224*factor, 224*factor))
                        crop = crop.convert("RGB")
                        crop = crop.resize((224, 224))
                
                        # predict
                        image_tensor = preprocess(crop)
                        image_tensor.unsqueeze_(0)
                        image_tensor = image_tensor.to('cuda')
                        model = model.to('cuda')
                        model.eval()
                        
                        with torch.no_grad():
                            output = model(image_tensor)
                            _, pred = torch.max(output, 1)
                            
                        pred = pred.item()
                
                        TISSUE.append(pred)

In [ ]:
DF = pd.DataFrame({
    'tile': TILE,
    'pathology': PATHOLOGY,
    'width(tile)': WIDTH_TILE,
    'height(tile)': HEIGHT_TILE,
    'x(tile)': X_TILE,
    'y(tile)': Y_TILE,
    'tissue': TISSUE
})

os.mkdir('./WSI_tile_classification')
for i in range(0, len(pathology_list_index)):
    dataframe = DF[DF['pathology'] == pathology_list_index[i]]
    dataframe.to_csv('./WSI_tile_classification/' + pathology_list_index[i] + '_tile_classification.csv', index=False)

## Mapping

In [ ]:
WSI_TILE_CLASSIFICATION_PATH = './WSI_tile_classification'
filename = os.listdir(WSI_TILE_CLASSIFICATION_PATH)
#print(len(filename))

### tumor

In [ ]:
os.mkdir('./tum_mapping')

for i in range(0, len(filename)):
    # Read csv
    dataframe = pd.read_csv(os.path.join(WSI_TILE_CLASSIFICATION_PATH, filename[i]))
    pathology = dataframe.at[0, 'pathology']
    width = dataframe.at[0, 'width(tile)']
    height = dataframe.at[0, 'height(tile)']
    
    # Create blank image
    image = Image.new('L', (width, height), 0)
    image_array = np.array(image)
    rows, cols = image_array.shape
    
    # Color tum block
    number_of_tile = len(dataframe.index)
    x_list = list(dataframe['x(tile)'])
    y_list = list(dataframe['y(tile)'])
    tissue_list = list(dataframe['tissue'])
    
    for j in range(0, number_of_tile):
        if tissue_list[j] == 8:
            image_array[y_list[j]-1, x_list[j]-1] = 255
    
    image_mapping = Image.fromarray(image_array)
    image_mapping.save(os.path.join('./tum_mapping', pathology+'.png'))

#### closing operation

In [ ]:
TUM_MAPPING_PATH = './tum_mapping'
TMfilename = os.listdir(TUM_MAPPING_PATH)

In [ ]:
os.mkdir('./tum_closed')

for i in range(0, len(TMfilename)):
    # Read image
    binary = cv2.imread(os.path.join(TUM_MAPPING_PATH, TMfilename[i]), cv2.IMREAD_GRAYSCALE)
    
    # Closing : connect objects that were mistakenly divided into many small pieces
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5)) # Structure Element
    closed = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
    
    # Save image
    cv2.imwrite(os.path.join('./tum_closed', TMfilename[i][0:23]+'.png'), closed)

### lymphocyte

In [ ]:
os.mkdir('./lym_mapping')

for i in range(0, len(filename)):
    # Read csv
    dataframe = pd.read_csv(os.path.join(WSI_TILE_CLASSIFICATION_PATH, filename[i]))
    pathology = dataframe.at[0, 'pathology']
    width = dataframe.at[0, 'width(tile)']
    height = dataframe.at[0, 'height(tile)']
    
    # Create blank image
    image = Image.new('L', (width, height), 0)
    image_array = np.array(image)
    rows, cols = image_array.shape
    
    # Color lym block
    number_of_tile = len(dataframe.index)
    x_list = list(dataframe['x(tile)'])
    y_list = list(dataframe['y(tile)'])
    tissue_list = list(dataframe['tissue'])
    
    for j in range(0, number_of_tile):
        if tissue_list[j] == 3:
            image_array[y_list[j]-1, x_list[j]-1] = 255
    
    image_mapping = Image.fromarray(image_array)
    image_mapping.save(os.path.join('./lym_mapping', pathology+'.png'))

### stroma

In [ ]:
os.mkdir('./stroma_mapping')

for i in range(0, len(filename)):
    # Read csv
    dataframe = pd.read_csv(os.path.join(WSI_TILE_CLASSIFICATION_PATH, filename[i]))
    pathology = dataframe.at[0, 'pathology']
    width = dataframe.at[0, 'width(tile)']
    height = dataframe.at[0, 'height(tile)']
    
    # Create blank image
    image = Image.new('L', (width, height), 0)
    image_array = np.array(image)
    rows, cols = image_array.shape
    
    # Color stroma block
    number_of_tile = len(dataframe.index)
    x_list = list(dataframe['x(tile)'])
    y_list = list(dataframe['y(tile)'])
    tissue_list = list(dataframe['tissue'])
    
    for j in range(0, number_of_tile):
        if tissue_list[j] == 7:
            image_array[y_list[j]-1, x_list[j]-1] = 255
    
    image_mapping = Image.fromarray(image_array)
    image_mapping.save(os.path.join('./stroma_mapping', pathology+'.png'))

#### closing operation

In [ ]:
STROMA_MAPPING_PATH = './stroma_mapping'
SMfilename = os.listdir(STROMA_MAPPING_PATH)

In [ ]:
os.mkdir('./stroma_closed')

for i in range(0, len(SMfilename)):
    # Read image
    binary = cv2.imread(os.path.join(STROMA_MAPPING_PATH, SMfilename[i]), cv2.IMREAD_GRAYSCALE)
    
    # Closing : connect objects that were mistakenly divided into many small pieces
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5)) # Structure Element
    closed = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
    
    # Save image
    cv2.imwrite(os.path.join('./stroma_closed', SMfilename[i][0:23]+'.png'), closed)

# Segmentation features

In [ ]:
pathology = []

for i in range(0, len(TMfilename)):
    pathology.append(TMfilename[i][0:23])

## max_tumor_area

In [ ]:
max_tumor_area = []

tum_number = []
tum_labels = []
tum_stats = []

for i in range(0, len(pathology)):
    # Read image
    closed = cv2.imread(os.path.join('./tum_closed', pathology[i]+'.png'), cv2.IMREAD_GRAYSCALE)
    
    # Connected component analysis
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(closed, connectivity=8, ltype=cv2.CV_32S)
    tum_number.append(num_labels-1)
    tum_labels.append(labels)
    tum_stats.append(stats) # x,y,width,height,area
    
    # max_tumor_area
    if num_labels > 1:
        max_tumor_area_label = np.argmax(stats[1:,4])+1
        max_tumor_area.append(stats[max_tumor_area_label, 4])
    else: 
        max_tumor_area.append(0)

## lymphocyte_inside_tumor

In [ ]:
lymphocyte_inside_tumor = []

for i in range(0, len(pathology)):
    # Read image
    tum_array = cv2.imread(os.path.join('./tum_closed', pathology[i]+'.png'), cv2.IMREAD_GRAYSCALE)
    lym_array = cv2.imread(os.path.join('./lym_mapping', pathology[i]+'.png'), cv2.IMREAD_GRAYSCALE)
    
    # lymphocyte_inside_tumor
    lit = 0
    for j in range(0, tum_array.shape[0]):
        for k in range(0, tum_array.shape[1]):
            if tum_array[j,k] == 255 and lym_array[j,k] == 255:
                lit = lit + 1
                
    lymphocyte_inside_tumor.append(lit)

## lymphocyte_around_tumor 

In [ ]:
lymphocyte_around_tumor = []

for i in range(0, len(pathology)):
    stats = copy.deepcopy(tum_stats[i]) # x,y,width,height,area
    tum_label = copy.deepcopy(tum_labels[i]) # 0,1,2...
    lym_array = cv2.imread(os.path.join('./lym_mapping', pathology[i]+'.png'), cv2.IMREAD_GRAYSCALE) # 0,255
    check_array = np.zeros(lym_array.shape, np.uint8)
    
    # establish check_array
    for j in range(1, tum_number[i]+1):
        # change the value of tumor j to 255, others to 0
        tumor = copy.deepcopy(tum_label)
        tumor[tumor != j] = 0
        tumor[tumor == j] = 255
        
        check_size = [20,20]
        
        for k in range(0, tumor.shape[0]):
            for l in range(0, tumor.shape[1]):
                if tumor[k,l] == 255:
                    y_lower = k - check_size[0]
                    if y_lower < 0:
                        y_lower = 0
                    y_upper = k + check_size[0]
                    if y_upper > tumor.shape[0]-1:
                        y_upper = tumor.shape[0]-1
                    
                    x_lower = l - check_size[1]
                    if x_lower < 0:
                        x_lower = 0
                    x_upper = l + check_size[1]
                    if x_upper > tumor.shape[1]-1:
                        x_upper = tumor.shape[1]-1
                    
                    for m in range(y_lower, y_upper+1):
                        for n in range(x_lower, x_upper+1):
                            check_array[m,n] = 255
    
    check_array[tum_label != 0] = 0
    
    # lymphocyte_around_tumor
    lat = 0
    for j in range(0, check_array.shape[0]):
        for k in range(0, check_array.shape[1]):
            if check_array[j,k] == 255 and lym_array[j,k] == 255:
                lat = lat + 1
    
    lymphocyte_around_tumor.append(lat)

## around_inside_ratio

In [ ]:
around_inside_ratio = []

for i in range(0, len(pathology)):
    around_inside_ratio.append((lymphocyte_around_tumor[i]+1) / (lymphocyte_inside_tumor[i]+1))

## total_stroma_area

In [ ]:
total_stroma_area = []

for i in range(0, len(pathology)):
    # Read image
    closed = cv2.imread(os.path.join('./stroma_closed', pathology[i]+'.png'), cv2.IMREAD_GRAYSCALE)
    
    # total_stroma_area
    total_stroma_area.append(np.count_nonzero(closed))

## OS

In [ ]:
survival = pd.read_csv('./survival_COAD_survival.csv', index_col = 'sample')

OS = []
OS_time = []

for i in range(0, len(pathology)):
    OS.append(survival.at[pathology[i][0:15], 'OS'])
    OS_time.append(survival.at[pathology[i][0:15], 'OS.time'])

## Dataframe

In [ ]:
df = pd.DataFrame({
    'pathology': pathology,
    'max_tumor_area': max_tumor_area,
    'lymphocyte_inside_tumor': lymphocyte_inside_tumor,
    'lymphocyte_around_tumor': lymphocyte_around_tumor,
    'around_inside_ratio': around_inside_ratio,
    'total_stroma_area': total_stroma_area,
    'OS': OS,
    'OS.time': OS_time
})

# Discretization

In [ ]:
segmentation_feature = ['max_tumor_area',
                        'lymphocyte_inside_tumor',
                        'lymphocyte_around_tumor',
                        'around_inside_ratio',
                        'total_stroma_area']

cutpoint = [11854, 245, 388, 0.8581315, 7324]

for i in range(0, len(pathology)):
    for j in range(0, len(segmentation_feature)):
        if df.at[i, segmentation_feature[j]] <= cutpoint[j]:
            df.at[i, segmentation_feature[j]] = 0
        else:
            df.at[i, segmentation_feature[j]] = 1

df.to_csv('./segmentation_features.csv', index=False)